In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import logit
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def generate_candidate_pair_data(n_pairs):
    """
    Generate realistic paired comparison data based on actual hiring statistics
    
    Parameters:
    -----------
    n_pairs : int
        Number of candidate pairs
        
    Returns:
    --------
    List of candidate pairs with attribute differences
    """
    
    pairs = []
    
    for pair_id in range(n_pairs):
        # Generate two candidates based on actual hiring data distributions
        candidates = []
        
        for candidate_num in range(2):
            # Basic demographics (from summary stats)
            female = np.random.binomial(1, 0.5)
            
            # Internship experience
            internship_exp = np.random.binomial(1, 0.483)
            
            # Certificates (0-4, following actual distribution)
            certificates = np.random.choice(range(5), p=[0.1, 0.25, 0.25, 0.25, 0.15])
            
            # University ranking (simplified to numeric for easier difference calculation)
            university_values = {
                'etc': 0,  # baseline
                'domestic_top14_22': 0.897,
                'international_middle_low': 1.136,
                'domestic_top10_13': 1.309,
                'domestic_top9_female': 1.400,
                'domestic_top6_8': 1.881,
                'domestic_top4_5': 2.348,
                'international_high': 2.525,
                'domestic_top3': 3.525
            }
            university_categories = list(university_values.keys())
            university_probs = [0.35, 0.05, 0.02, 0.18, 0.02, 0.15, 0.12, 0.03, 0.08]
            university_cat = np.random.choice(university_categories, p=university_probs)
            university_score = university_values[university_cat]
            
            # GPA (from summary stats, no quadratic term)
            gpa = np.random.normal(80.934, 7.489)
            gpa = np.clip(gpa, 36.4, 100.0)
            
            candidates.append({
                'female': female,
                'internship_exp': internship_exp,
                'certificates': certificates,
                'university_score': university_score,
                'gpa': gpa
            })
        
        # Calculate differences (Candidate 1 - Candidate 2)
        delta_gender = (1 - candidates[0]['female']) - (1 - candidates[1]['female'])  # male=1, female=0
        delta_internship_exp = candidates[0]['internship_exp'] - candidates[1]['internship_exp']
        delta_certificates = candidates[0]['certificates'] - candidates[1]['certificates']
        delta_university = candidates[0]['university_score'] - candidates[1]['university_score']
        delta_gpa = candidates[0]['gpa'] - candidates[1]['gpa']
        
        pairs.append({
            'pair_id': pair_id,
            'delta_gender': delta_gender,
            'delta_internship_exp': delta_internship_exp,
            'delta_certificates': delta_certificates,
            'delta_university': delta_university,
            'delta_gpa': delta_gpa,
            'candidate1': candidates[0],
            'candidate2': candidates[1]
        })
    
    return pairs

def create_balanced_groups_with_responsibility(n_participants):
    """
    Create balanced groups with proper 2x2 design (AI × Responsibility)
    Fixed to create balanced 2x2 design with equal allocation to all 4 conditions
    
    Parameters:
    -----------
    n_participants : int
        Total number of participants (must be multiple of 12)
        
    Returns:
    --------
    List of groups with balanced AI and responsibility assignment
    """
    if n_participants % 12 != 0:
        raise ValueError("Number of participants must be multiple of 12 for balanced design")
    
    n_groups = n_participants // 3
    
    # Ensure we can create balanced 2x2 design
    if n_groups % 4 != 0:
        raise ValueError("Number of groups must be multiple of 4 for balanced 2x2 design")
    
    groups_per_condition = n_groups // 4  # 25% each condition
    
    # Generate participants with balanced gender (50-50)
    participants = []
    for i in range(n_participants):
        participants.append({
            'participant_id': i,
            'gender': i % 2,  # 0=male, 1=female, alternating for balance
        })
    
    # Separate by gender
    males = [p for p in participants if p['gender'] == 0]
    females = [p for p in participants if p['gender'] == 1]
    
    # Shuffle for randomization
    np.random.shuffle(males)
    np.random.shuffle(females)
    
    # Create 4 types of groups with equal proportions (25% each)
    compositions = [
        ('MMM', 3, 0),   # 3 males, 0 females - homogeneous
        ('MMF', 2, 1),   # 2 males, 1 female - mixed
        ('MFF', 1, 2),   # 1 male, 2 females - mixed  
        ('FFF', 0, 3)    # 0 males, 3 females - homogeneous
    ]
    
    groups = []
    male_idx = 0
    female_idx = 0
    group_id = 0
    
    # Create 2x2 conditions: AI (Yes/No) × Responsibility (High/Low)
    conditions = [
        (True, True),    # AI=1, High_Resp=1
        (True, False),   # AI=1, High_Resp=0  
        (False, True),   # AI=0, High_Resp=1
        (False, False)   # AI=0, High_Resp=0
    ]
    
    # For each composition type (MMM, MMF, MFF, FFF)
    for comp_name, n_males, n_females in compositions:
        groups_per_comp = groups_per_condition  # Equal groups per composition
        
        # Within each composition, assign groups to 4 conditions equally
        condition_idx = 0
        for _ in range(groups_per_comp):
            group_members = []
            
            # Add required males
            for _ in range(n_males):
                if male_idx < len(males):
                    group_members.append(males[male_idx])
                    male_idx += 1
            
            # Add required females  
            for _ in range(n_females):
                if female_idx < len(females):
                    group_members.append(females[female_idx])
                    female_idx += 1
            
            # Assign to current condition
            group_has_ai, group_high_responsibility = conditions[condition_idx]
            condition_idx = (condition_idx + 1) % 4  # Cycle through conditions
            
            # Determine if group is mixed or homogeneous
            is_mixed = comp_name in ['MMF', 'MFF']
            composition_type = 'mixed' if is_mixed else 'homogeneous'
            
            groups.append({
                'group_id': group_id,
                'members': group_members,
                'composition': composition_type,
                'composition_detailed': comp_name,
                'has_ai': group_has_ai,
                'high_responsibility': group_high_responsibility
            })
            
            group_id += 1
    
    # Shuffle groups to randomize order
    np.random.shuffle(groups)
    
    # Verify balanced assignment
    ai_counts = {'AI=1,Resp=1': 0, 'AI=1,Resp=0': 0, 'AI=0,Resp=1': 0, 'AI=0,Resp=0': 0}
    for group in groups:
        key = f"AI={'1' if group['has_ai'] else '0'},Resp={'1' if group['high_responsibility'] else '0'}"
        ai_counts[key] += 1
    
    return groups

def calculate_selection_probability(pair_data, has_ai, is_group, group_composition, 
                                                is_high_responsibility, effect_sizes, random_effect):
    """
    재구성된 확률 계산: Individual vs Group 전체 비교 시 group_reduces_bias 효과 보장
    """
    
    # Base coefficients
    coefficients = {
        'intercept': 0,
        'gender': 1.519,  
        'internship_exp': 0.360,
        'certificates': 0.169,
        'university': 1.0,
        'gpa': 0.335/10
    }
    
    # Base log odds from attribute differences
    log_odds = coefficients['intercept']
    log_odds += coefficients['gender'] * pair_data['delta_gender']
    log_odds += coefficients['internship_exp'] * pair_data['delta_internship_exp']
    log_odds += coefficients['certificates'] * pair_data['delta_certificates']
    log_odds += coefficients['university'] * pair_data['delta_university']
    log_odds += coefficients['gpa'] * pair_data['delta_gpa']
    
    # Add random effect
    log_odds += random_effect
    
    if is_group:
        # ===== 핵심 변경: 모든 그룹의 평균 효과가 group_reduces_bias가 되도록 구성 =====
        
        # 1. 기본 그룹 효과 (모든 그룹 조건의 중심점)
        base_group_effect = effect_sizes['group_reduces_bias']
        log_odds += base_group_effect * pair_data['delta_gender']
        
        # 2. 각 조건별 추가/차감 효과 (평균이 0이 되도록 설계)
        
        # AI 효과 (AI 그룹은 +, No AI 그룹은 -)
        if has_ai:
            log_odds += effect_sizes['ai_additional_effect'] * pair_data['delta_gender']
        else:
            log_odds += (-effect_sizes['ai_additional_effect']) * pair_data['delta_gender']
        
        # 그룹 구성 효과 (Mixed는 +, Homogeneous는 -)  
        if group_composition == 'mixed':
            log_odds += effect_sizes['mixed_additional_effect'] * pair_data['delta_gender']
        else:  # homogeneous
            log_odds += (-effect_sizes['mixed_additional_effect']) * pair_data['delta_gender']
        
        # 책임감 효과 (High는 +, Low는 -)
        if is_high_responsibility:
            log_odds += effect_sizes['responsibility_additional_effect'] * pair_data['delta_gender']
        else:
            log_odds += (-effect_sizes['responsibility_additional_effect']) * pair_data['delta_gender']
        
        # 상호작용 효과들 (여전히 조건부)
        if has_ai and group_composition == 'mixed':
            log_odds += effect_sizes['mixed_ai_synergy'] * pair_data['delta_gender']
        else:
            log_odds += (-effect_sizes['mixed_ai_synergy'] * pair_data['delta_gender'])
        
        if has_ai and is_high_responsibility:
            log_odds += effect_sizes['ai_responsibility_synergy'] * pair_data['delta_gender']
        else:
            log_odds += (-effect_sizes['ai_responsibility_synergy'] * pair_data['delta_gender'])
    
    # Convert to probability
    prob = 1 / (1 + np.exp(-log_odds))
    return np.clip(prob, 1e-10, 1-1e-10)


def generate_study2_data_with_responsibility(n_participants, pairs_per_participant_indiv, pairs_per_participant_group, effect_sizes):
    """
    Generate data for Study 2 with individual and group phases, with separate decision counts
    
    Parameters:
    -----------
    n_participants : int
        Number of participants (must be divisible by 12)
    pairs_per_participant_indiv : int
        Number of pairs each participant evaluates individually
    pairs_per_participant_group : int
        Number of pairs each group evaluates together
    effect_sizes : dict
        Effect size parameters
    """
    data_rows = []
    
    # Generate candidate pairs for individual phase
    individual_pairs = generate_candidate_pair_data(pairs_per_participant_indiv)
    
    # Generate DIFFERENT candidate pairs for group phase
    group_pairs = generate_candidate_pair_data(pairs_per_participant_group)
    
    # Create balanced groups with responsibility
    groups = create_balanced_groups_with_responsibility(n_participants)
    
    # Create participant info lookup
    participant_info = {}
    for group in groups:
        for member in group['members']:
            participant_info[member['participant_id']] = {
                'gender': member['gender'],
                'has_ai': group['has_ai'],
                'group_id': group['group_id'],
                'group_composition': group['composition'],
                'high_responsibility': group['high_responsibility']
            }
    
    # Generate random effects for participants and groups
    # These remain constant across all decisions for the same unit
    participant_random_effects = {}
    group_random_effects = {}
    
    # Generate individual random effects (one per participant)
    sigma_within = np.pi**2 / 3 
    target_icc_indiv = 0.25
    target_icc_group = 0.1
    sigma_participant_indiv = np.sqrt(target_icc_indiv * sigma_within / (1 - target_icc_indiv)) 
    sigma_participant_group = np.sqrt(target_icc_group * sigma_within / (1 - target_icc_group)) 

    participant_random_effects = {}
    for participant_id in range(n_participants):
        participant_random_effects[participant_id] = np.random.normal(0, sigma_participant_indiv)

    for group in groups:
        group_random_effects[group['group_id']] = np.random.normal(0, sigma_participant_group)
    
    # Individual phase - each participant evaluates individual_pairs
    for participant_id in range(n_participants):
        info = participant_info[participant_id]
        
        for pair in individual_pairs:
            prob = calculate_selection_probability(
                pair, info['has_ai'], is_group=False, group_composition=None, 
                is_high_responsibility=None,  # No responsibility in individual phase
                effect_sizes=effect_sizes, 
                random_effect=participant_random_effects[participant_id]  # Use consistent random effect
            )
            y_individual = np.random.binomial(1, prob)
            
            data_rows.append({
                'participant_id': participant_id,
                'pair_id': f"ind_{pair['pair_id']}",  # Mark as individual pair
                'y': y_individual,
                'delta_gender': pair['delta_gender'],
                'delta_internship_exp': pair['delta_internship_exp'],
                'delta_certificates': pair['delta_certificates'],
                'delta_university': pair['delta_university'],
                'delta_gpa': pair['delta_gpa'],
                'has_ai': info['has_ai'],
                'is_group': 0,
                'group_composition': 'individual',
                'high_responsibility': None,  # No responsibility in individual phase
                'participant_gender': info['gender'],
                'group_id': info['group_id']
            })
    
    # Group phase - each group evaluates group_pairs (NEW pairs)
    for group in groups:
        group_members = [member['participant_id'] for member in group['members']]
        group_composition = group['composition']
        group_has_ai = group['has_ai']
        group_high_responsibility = group['high_responsibility']
        
        # Group decisions on NEW pairs (not the same as individual pairs)
        for pair in group_pairs:
            # Use group probability model with consistent group random effect
            prob = calculate_selection_probability(
                pair, group_has_ai, is_group=True, 
                group_composition=group_composition, 
                is_high_responsibility=group_high_responsibility,
                effect_sizes=effect_sizes,
                random_effect=group_random_effects[group['group_id']]  # Use consistent group random effect
            )
            y_group = np.random.binomial(1, prob)
            
            # Add group decision for each member (same decision, different rows for analysis)
            for member in group_members:
                member_gender = participant_info[member]['gender']
                
                data_rows.append({
                    'participant_id': member,
                    'pair_id': f"grp_{pair['pair_id']}",  # Mark as group pair
                    'y': y_group,
                    'delta_gender': pair['delta_gender'],
                    'delta_internship_exp': pair['delta_internship_exp'],
                    'delta_certificates': pair['delta_certificates'],
                    'delta_university': pair['delta_university'],
                    'delta_gpa': pair['delta_gpa'],
                    'has_ai': group_has_ai,
                    'is_group': 1,
                    'group_composition': group_composition,
                    'high_responsibility': group_high_responsibility,
                    'participant_gender': member_gender,
                    'group_id': group['group_id']
                })
    
    return pd.DataFrame(data_rows)

def test_hypothesis_h5(df, alpha=0.05):
    """Test H5: Group decision reduces gender bias"""
    try:
        formula = """
            y ~ delta_gender + delta_internship_exp + delta_certificates + 
              delta_university + delta_gpa + is_group + 
              delta_gender:is_group
        """
        
        model = logit(formula, data=df).fit(
            cov_type='cluster', 
            cov_kwds={'groups': df['participant_id']},
            disp=0, maxiter=100
        )
        
        # Test if group reduces gender bias (negative interaction)
        p_val = model.pvalues.get('delta_gender:is_group', 1.0)
        return p_val < alpha
        
    except:
        return False

def test_hypothesis_h6(df, alpha=0.05):
    """Test H6: Fair AI reduces gender bias in group decisions"""
    try:
        # Restrict to group decisions only
        df_group = df[df['is_group'] == 1].copy()
        
        formula = """
            y ~ delta_gender + delta_internship_exp + delta_certificates + 
              delta_university + delta_gpa + has_ai + 
              delta_gender:has_ai
        """
        
        model = logit(formula, data=df_group).fit(
            cov_type='cluster', 
            cov_kwds={'groups': df_group['group_id']},
            disp=0, maxiter=100
        )
        
        # Test if AI reduces gender bias in groups (negative interaction)
        p_val = model.pvalues.get('delta_gender:has_ai', 1.0)
        return p_val < alpha
        
    except:
        return False

def test_hypothesis_h7(df, alpha=0.05):
    """Test H7: Three-way interaction (Group × AI × Gender)"""
    try:
        formula = """
            y ~ delta_gender + delta_internship_exp + delta_certificates + 
              delta_university + delta_gpa + is_group + has_ai +
              delta_gender:is_group + delta_gender:has_ai + is_group:has_ai +
              delta_gender:is_group:has_ai
        """
        
        model = logit(formula, data=df).fit(
            cov_type='cluster', 
            cov_kwds={'groups': df['participant_id']},
            disp=0, maxiter=100
        )
        
        # Test three-way interaction
        p_val = model.pvalues.get('delta_gender:is_group:has_ai', 1.0)
        return p_val < alpha
        
    except:
        return False   

def test_hypothesis_h8(df, alpha=0.05):
    """
    Test H8: Mixed-gender groups exhibit lowest gender bias (compared to homogeneous groups)
    """
    try:
        # Restrict to group decisions only
        df_group = df[df['is_group'] == 1].copy()
        
        # Create dummy for mixed vs homogeneous
        df_group['is_mixed'] = (df_group['group_composition'] == 'mixed').astype(int)
        
        formula = """
            y ~ delta_gender + delta_internship_exp + delta_certificates + 
              delta_university + delta_gpa + is_mixed +
              delta_gender:is_mixed
        """        
        model = logit(formula, data=df_group).fit(
            cov_type='cluster', 
            cov_kwds={'groups': df_group['group_id']},
            disp=0, maxiter=100
        )
        
        # H8: Test if mixed groups have lower bias (negative coefficient for delta_gender:is_mixed)
        if 'delta_gender:is_mixed' in model.pvalues:
            h8_pval = model.pvalues['delta_gender:is_mixed']  # Fixed: removed is_group
            h8_coef = model.params['delta_gender:is_mixed']
            return h8_pval < alpha and h8_coef < 0  # Negative = less bias
        
        return False
        
    except Exception as e:
        print(f"H8 test failed: {e}")
        return False

def test_hypothesis_h9(df, alpha=0.05):
    """
    Test H9: Mixed-gender groups benefit most from AI interventions
    """
    try:
        # Restrict to group decisions only
        df_group = df[df['is_group'] == 1].copy()
        
        # Create dummy for mixed vs homogeneous
        df_group['is_mixed'] = (df_group['group_composition'] == 'mixed').astype(int)
        
        formula = """
            y ~ delta_gender + delta_internship_exp + delta_certificates + 
              delta_university + delta_gpa + has_ai + is_mixed +
              delta_gender:is_mixed + delta_gender:has_ai +
              delta_gender:is_mixed:has_ai
        """
        
        model = logit(formula, data=df_group).fit(
            cov_type='cluster', 
            cov_kwds={'groups': df_group['group_id']},
            disp=0, maxiter=100
        )
        
        # H9: Test if mixed groups benefit more from AI (negative three-way interaction)
        if 'delta_gender:is_mixed:has_ai' in model.pvalues:
            h9_pval = model.pvalues['delta_gender:is_mixed:has_ai']
            return h9_pval < alpha
        
        return False
        
    except:
        return False

def test_hypothesis_h10(df, alpha=0.05):
    """
    Test H10: Responsibility sharing moderates AI effectiveness in reducing gender bias
    """
    try:
        # Restrict to group decisions only
        df_group = df[df['is_group'] == 1].copy()
        
        # Remove rows where high_responsibility is None (shouldn't happen in group phase)
        df_group = df_group.dropna(subset=['high_responsibility'])
        
        if len(df_group) == 0:
            return False
        
        # Convert to numeric
        df_group['high_responsibility'] = df_group['high_responsibility'].astype(int)
        
        formula = """
            y ~ delta_gender + delta_internship_exp + delta_certificates + 
              delta_university + delta_gpa + has_ai + high_responsibility +
              delta_gender:has_ai + delta_gender:high_responsibility +
              delta_gender:has_ai:high_responsibility
        """
        
        model = logit(formula, data=df_group).fit(
            cov_type='cluster', 
            cov_kwds={'groups': df_group['group_id']},
            disp=0, maxiter=100
        )
        
        # H10: Test if responsibility moderates AI effectiveness (three-way interaction)
        if 'delta_gender:has_ai:high_responsibility' in model.pvalues:
            h10_pval = model.pvalues['delta_gender:has_ai:high_responsibility']
            return h10_pval < alpha
        
        return False
        
    except:
        return False

def simulate_study2_power_analysis_with_h10(n_participants_list, pairs_per_participant_indiv, pairs_per_participant_group, effect_sizes, n_sim=1000, alpha=0.05):
    """
    Power analysis simulation for Study 2 including H10 with separate individual and group decision counts
    """
    
    results = {
        'n_participants': n_participants_list,
        'power_h5': [],  # Group reduces bias
        'power_h6': [],  # AI reduces bias in groups
        'power_h7': [],  # Three-way interaction
        'power_h8': [],  # Mixed groups have lowest bias
        'power_h9': [],  # Mixed groups benefit most from AI
        'power_h10': [], # Responsibility moderates AI effectiveness
        'convergence_failures': [],
        'exceptions': []
    }
    
    print("Study 2 Power Analysis with H10")
    print(f"Individual pairs per participant: {pairs_per_participant_indiv}")
    print(f"Group pairs per group: {pairs_per_participant_group}")
    print(f"Group size: 3 participants")
    print("Testing hypotheses:")
    print("  H5: Group decision reduces gender bias")
    print("  H6: Fair AI reduces gender bias in group decisions") 
    print("  H7: Group × AI × Gender three-way interaction")
    print("  H8: Mixed-gender groups have lowest bias")
    print("  H9: Mixed-gender groups benefit most from AI")
    print("  H10: Responsibility sharing moderates AI effectiveness")
    
    for n in n_participants_list:
        if n % 12 != 0:
            print(f"WARNING: {n} participants cannot form balanced 4-way groups. Adjusting to {n//12*12}")
            n = n // 12 * 12
            
        n_groups = n // 3
        individual_decisions = n * pairs_per_participant_indiv
        group_decisions = n_groups * pairs_per_participant_group
        total_decisions = individual_decisions + group_decisions
        
        print(f"\nRunning simulation for {n} participants ({n_groups} groups)...")
        print(f"  Individual decisions: {individual_decisions:,}")
        print(f"  Group decisions: {group_decisions:,}")
        print(f"  Total decisions: {total_decisions:,}")
        print(f"  Group composition: {n_groups//4} groups each of MMM/MMF/MFF/FFF")
        print(f"  AI condition: {n_groups//2} AI groups, {n_groups//2} no-AI groups")
        print(f"  Responsibility: {n_groups//2} high-responsibility groups, {n_groups//2} low-responsibility groups")
        
        power_counts = {'h5': 0, 'h6': 0, 'h7': 0, 'h8': 0, 'h9': 0, 'h10': 0}
        convergence_fails = 0
        exceptions = 0
        
        for sim in tqdm(range(n_sim), desc=f"n={n}", leave=False):
            try:
                # Generate data
                df = generate_study2_data_with_responsibility(n, pairs_per_participant_indiv, pairs_per_participant_group, effect_sizes)
                
                # Convert to float
                for col in ['has_ai', 'is_group', 'delta_gender']:
                    df[col] = df[col].astype(float)

                # Test each hypothesis
                if test_hypothesis_h5(df, alpha):
                    power_counts['h5'] += 1
                    
                if test_hypothesis_h6(df, alpha):
                    power_counts['h6'] += 1
                    
                if test_hypothesis_h7(df, alpha):
                    power_counts['h7'] += 1
                    
                if test_hypothesis_h8(df, alpha):
                    power_counts['h8'] += 1
                    
                if test_hypothesis_h9(df, alpha):
                    power_counts['h9'] += 1
                    
                if test_hypothesis_h10(df, alpha):
                    power_counts['h10'] += 1
                    
            except Exception as e:
                if "Singular matrix" in str(e) or "convergence" in str(e).lower():
                    convergence_fails += 1
                else:
                    exceptions += 1
                continue
        
        # Calculate power
        valid_sims = n_sim - exceptions
        
        if valid_sims > 0:
            results['power_h5'].append(power_counts['h5'] / valid_sims)
            results['power_h6'].append(power_counts['h6'] / valid_sims)
            results['power_h7'].append(power_counts['h7'] / valid_sims)
            results['power_h8'].append(power_counts['h8'] / valid_sims)
            results['power_h9'].append(power_counts['h9'] / valid_sims)
            results['power_h10'].append(power_counts['h10'] / valid_sims)
        else:
            results['power_h5'].append(0)
            results['power_h6'].append(0)
            results['power_h7'].append(0)
            results['power_h8'].append(0)
            results['power_h9'].append(0)
            results['power_h10'].append(0)
            
        results['convergence_failures'].append(convergence_fails / n_sim)
        results['exceptions'].append(exceptions / n_sim)
    
    return results

def plot_study2_power_curves_with_h10(results, pairs_per_participant_indiv, pairs_per_participant_group):
    """Plot power curves for Study 2 hypotheses including H10"""
    plt.figure(figsize=(20, 16))
    
    hypotheses = [
        ('power_h5', 'H5: Group Reduces Gender Bias', 'blue'),
        ('power_h6', 'H6: AI Reduces Bias in Groups', 'green'),
        ('power_h7', 'H7: Group × AI × Gender Interaction', 'red'),
        ('power_h8', 'H8: Mixed Groups Have Lowest Bias', 'purple'),
        ('power_h9', 'H9: Mixed Groups Benefit Most from AI', 'orange'),
        ('power_h10', 'H10: Responsibility Moderates AI Effect', 'brown')
    ]
    
    for i, (power_key, title, color) in enumerate(hypotheses):
        plt.subplot(2, 3, i+1)
        plt.plot(results['n_participants'], results[power_key], 'o-', 
                linewidth=2, markersize=6, color=color)
        plt.axhline(0.8, color='orange', linestyle='--', alpha=0.7, label='Power = 0.8')
        plt.axhline(0.95, color='red', linestyle='--', alpha=0.7, label='Power = 0.95')
        plt.title(f'{title}\n(Individual: {pairs_per_participant_indiv}, Group: {pairs_per_participant_group} pairs)', fontsize=11)
        plt.xlabel('Number of Participants')
        plt.ylabel('Estimated Power')
        plt.ylim(0, 1)
        plt.grid(True, alpha=0.3)
        plt.legend()
    
    plt.tight_layout()
    plt.show()

def find_study2_sample_requirements_with_h10(results, pairs_per_participant_indiv, pairs_per_participant_group):
    """Find sample size requirements for Study 2 hypotheses including H10"""
    
    hypotheses = [
        ('power_h5', 'H5: Group Reduces Gender Bias'),
        ('power_h6', 'H6: AI Reduces Bias in Groups'),
        ('power_h7', 'H7: Group × AI × Gender Interaction'),
        ('power_h8', 'H8: Mixed Groups Have Lowest Bias'),
        ('power_h9', 'H9: Mixed Groups Benefit Most from AI'),
        ('power_h10', 'H10: Responsibility Moderates AI Effect')
    ]
    
    for target_power in [0.8, 0.95]:
        print(f"\n=== {target_power*100}% Power Requirements ===")
        
        for power_key, hypothesis_name in hypotheses:
            found = False
            for i, power in enumerate(results[power_key]):
                if power >= target_power:
                    n_participants = results['n_participants'][i]
                    n_groups = n_participants // 3
                    individual_decisions = n_participants * pairs_per_participant_indiv
                    group_decisions = n_groups * pairs_per_participant_group  
                    total_decisions = individual_decisions + group_decisions
                    print(f"{hypothesis_name}: {n_participants} participants "
                          f"({n_groups} groups, {total_decisions:,} total decisions)"
                          f" [Individual: {individual_decisions:,}, Group: {group_decisions:,}]")
                    found = True
                    break
            
            if not found:
                max_n = max(results['n_participants'])
                max_power = max(results[power_key])
                max_groups = max_n // 3
                max_individual = max_n * pairs_per_participant_indiv
                max_group_decisions = max_groups * pairs_per_participant_group
                max_total = max_individual + max_group_decisions
                print(f"{hypothesis_name}: >{max_n} participants needed "
                      f"(max observed power: {max_power:.3f}, would need >{max_total:,} decisions)")

# Example execution
if __name__ == "__main__":
    # Experimental design parameters - NOW SEPARATE
    pairs_per_participant_indiv = 10  # Each participant evaluates 10 pairs individually
    pairs_per_participant_group = 30   # Each group evaluates 30 pairs together (balances the decision counts)
    
    participants_range = list(range(60, 301, 60))  # Must be multiples of 12: 240, 360, 480
    
    # Adjust to multiples of 12 for balanced 4-way design
    participants_range = [n for n in participants_range if n % 12 == 0]
    if not participants_range:
        participants_range = [240, 360, 480]  # All multiples of 12
    
    # Define effect sizes - Reduced to avoid saturation effects
    # Individual effects should be smaller when combined
    effect_sizes = {
        'group_reduces_bias': -1.451,  # 계산된 기준점
        
        # 각 조건별 편차 (평균이 0이 되도록 설계)
        'ai_additional_effect': -0.542/2,          # 
        'mixed_additional_effect': -0.542/2,      #  small to medium effect cohed's d = 0.3 
        'responsibility_additional_effect': 0,  
        
        # 상호작용 효과들
        'mixed_ai_synergy': -1.451/2,           # Mixed × AI 추가 시너지 large effect
        'ai_responsibility_synergy': -1.451/2    # AI × High Responsibility 추가 시너지
    }

    
    print("=== STUDY 2 POWER ANALYSIS WITH H10 (BALANCED 4-WAY DESIGN) ===")
    print(f"Participants range: {participants_range} (multiples of 12)")
    print(f"Individual pairs per participant: {pairs_per_participant_indiv}")
    print(f"Group pairs per group: {pairs_per_participant_group}")
    print(f"Group composition: 25% each of MMM/MMF/MFF/FFF")
    print(f"  - Homogeneous groups: MMM + FFF (50%)")
    print(f"  - Mixed groups: MMF + MFF (50%)")
    print(f"AI condition: 50% groups AI, 50% groups no-AI")
    print(f"Responsibility condition: 50% groups high, 50% groups low")
    print("Effect sizes:")
    for key, value in effect_sizes.items():
        print(f"  {key}: {value:.3f}")
    print("\nHypotheses:")
    print("  H5: Group decision reduces gender bias")
    print("  H6: Fair AI reduces gender bias in group decisions")
    print("  H7: Group × AI × Gender three-way interaction")
    print("  H8: Mixed-gender groups exhibit lowest levels of gender bias")
    print("  H9: Mixed-gender groups benefit most from fair AI interventions")
    print("  H10: High responsibility groups benefit more from AI than low responsibility groups")
    
    # Run power analysis
    results = simulate_study2_power_analysis_with_h10(
        n_participants_list=participants_range,
        pairs_per_participant_indiv=pairs_per_participant_indiv,
        pairs_per_participant_group=pairs_per_participant_group,
        effect_sizes=effect_sizes,
        n_sim=1000,
        alpha=0.05
    )
    
    # Plot results
    plot_study2_power_curves_with_h10(results, pairs_per_participant_indiv, pairs_per_participant_group)
    
    # Find sample size requirements
    find_study2_sample_requirements_with_h10(results, pairs_per_participant_indiv, pairs_per_participant_group)
    
    # Print detailed results
    print(f"\n=== DETAILED RESULTS ===")
    for i, n in enumerate(results['n_participants']):
        n_groups = n // 3
        individual_decisions = n * pairs_per_participant_indiv
        group_decisions = n_groups * pairs_per_participant_group
        total_decisions = individual_decisions + group_decisions
        print(f"n={n:3d} ({n_groups:2d} groups, {total_decisions:4d} decisions): "
              f"H5={results['power_h5'][i]:.3f}, "
              f"H6={results['power_h6'][i]:.3f}, "
              f"H7={results['power_h7'][i]:.3f}, "
              f"H8={results['power_h8'][i]:.3f}, "
              f"H9={results['power_h9'][i]:.3f}, "
              f"H10={results['power_h10'][i]:.3f}")